# [ご参考] Doc2VecでWikipedia文書を使用し予測

巨大コーパスをもつWiki文書の一部を使用して、Wiki文書でボキャブラリを作成し、任意の質問文で学習／予測させます。

## (1) Wikipediaコンテンツファイルから全文書を抽出

<a href="31-Wikipedia-contents-csv.ipynb"><b>こちらの手順</b></a> にて、いったんローカルPCにCSVファイル化しておきます。



## (2) TaggedDocumentを生成

### (2-1) Wikipedia文書のTaggedDocumentを生成

Wikipedia文書のCSVファイルからデータフレームを読み込み、TaggedDocumentに変換します。

In [1]:
'''
    環境準備
'''
import sys
import os

import numpy as np
import pandas as pd
 
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [3]:
from gensim.models.doc2vec import TaggedDocument
# from learning.core.nlang import Nlang

def get_tagged_document_list(wiki_ids, wiki_contents):
    '''
        タグとコーパスをTaggedDocumentに設定
    '''
    tagged_document_list = []

    for index, _ in enumerate(wiki_ids):
        tag = wiki_ids[index]
        sentences = wiki_contents[index]

        # 品詞は落とさない様にする
        separated_sentences = Nlang_naive.split(sentences)
        words = separated_sentences.split()
        tagged_document_list.append(TaggedDocument(words=words, tags=[tag]))
        
    return tagged_document_list

In [4]:
'''
    Wikipedia文書から生成されたCSVを読み込み、
    配列を生成する
'''
extracted_dir_path = '/Users/makmorit/Documents/Development/Wikipedia/extracted'
csv_file_name = os.path.join(extracted_dir_path, 'Wikipedia-content.csv')
df = pd.read_csv(csv_file_name)

In [5]:
_wiki_ids = np.array(df['id'])
_wiki_titles = np.array(df['title'])
_wiki_contents = np.array(df['content'])

In [6]:
'''
    時間がかかるので一部のみ使用します（全量＝100万件）
'''
partial_cnt = 10000

def split_wiki_corpus(wiki_id, wiki_content, wiki_title):
    '''
        コーパスを分割

        予測処理のインプットとなる単語数に合わせるため、
        句読点でコーパスを分割する。
        ただし、この段階でIDをユニーク化すると
        分割されたコーパス間で意味が繋がらなくなるため、
        IDを [doc id]_[serial] 形式で付番する
    '''
    sub_ids = []
    sub_contents = []
    titles = []
    
    sub_index = 0
    for sub_content in wiki_content.split('。'):
        if len(sub_content) == 0:
            continue
        sub_id = '%d_%d' % (wiki_id, sub_index)
        sub_ids.append(sub_id)
        sub_contents.append(sub_content)
        titles.append(wiki_title)
        sub_index += 1
        
        if sub_index == 10:
            break

    return sub_ids, sub_contents, titles

wiki_ids = []
wiki_contents = []
wiki_titles = []

for index in range(partial_cnt):
    wiki_id = _wiki_ids[index]
    wiki_content = _wiki_contents[index]
    wiki_title = _wiki_titles[index]

    sub_ids, sub_contents, titles = split_wiki_corpus(wiki_id, wiki_content, wiki_title)

    wiki_ids.extend(sub_ids)
    wiki_contents.extend(sub_contents)
    wiki_titles.extend(titles)

In [7]:
wiki_tagged_document_list = get_tagged_document_list(wiki_ids, wiki_contents)

In [8]:
len(wiki_tagged_document_list)

87181

In [9]:
len(wiki_titles)

87181

### (2-2) Wikipedia文書とマイオペ文書の、両方のTaggedDocumentを合体させる

今回は合体させません

In [10]:
all_tagged_document_list = []
all_tagged_document_list.extend(wiki_tagged_document_list)
#all_tagged_document_list.extend(myope_tagged_document_list)

In [11]:
len(all_tagged_document_list)

87181

## (3) 学習実行

学習には５時間程度を要しました。

### (3-1) TaggedDocumentをインプットとして学習

In [12]:
from gensim.models.doc2vec import Doc2Vec

def doc2vec_model_path(dm):
    model_path = 'prototype/better_algorithm/doc2vec.wikipedia.PV%d.model' % dm
    return model_path

def build_vocabulary(tagged_document_list, dm=0, size=100):
    '''
        tagged_document_listは、
        Wikipedia文書のタグ付きコーパス
        (前述のtagged_document)が収容されたリスト。

        これを引数にして、
        ボキャブラリ生成を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
    '''
    model = Doc2Vec(dm=dm, size=size, window=20, min_count=1, iter=size*10, workers=100)
    model.build_vocab(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('build_vocabulary: model saved temporary')

    return model

def train(model, tagged_document_list, dm=0):
    '''
        tagged_documentが収容されたリストを引数にして、
        学習を実行します。
        
        dm=0でPV-DBOW、dm=1でPV-DMにより実行します。
        （今回は比較のため、両方で試す）
    '''
    ret = model.train(tagged_document_list)

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(dm))
    print('train: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [13]:
'''
    tagged_document_listを使用し、学習実行（PV-DBOW）
'''
model_dbow = build_vocabulary(all_tagged_document_list, dm=0, size=200)

build_vocabulary: model saved temporary


In [14]:
train(model_dbow, all_tagged_document_list, dm=0) 

train: document vector size=87181, return=3532258244


3532258244

In [15]:
'''
    tagged_document_listを使用し、学習実行（PV-DM）
'''
model_dm = build_vocabulary(all_tagged_document_list, dm=1, size=200)

build_vocabulary: model saved temporary


In [16]:
train(model_dm, all_tagged_document_list, dm=1) 

train: document vector size=87181, return=3532271999


3532271999

## (3) 予測実行

作成されたモデルを使用して予測を実行する関数になります。

In [17]:
from gensim import models

def predict(word, dm=0):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(dm))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

def get_title_from_tag(tag):
    '''
        tag に対応するWikipediaのタイトルを取得。

        ただし、合致したものがなければ、
        マイオペの id と判断し、
        id に対応する answer_id を取得する。
    '''
    for index, _ in enumerate(wiki_ids):
        if wiki_ids[index] == tag:
            return wiki_titles[index], wiki_contents[index]

    for index, _ in enumerate(ids):
        if ids[index] == tag:
            return answer_ids[index]
        
    return None

def print_predict_result(corpus, similarities):
    print(corpus)
    print("id, answer_id, similarity")

    for tag, similarity in similarities:
        print(tag, get_title_from_tag(tag), similarity)

In [18]:
len(wiki_titles)

87181

### (3-1) 適当な質問文で予測（PV-DBOW)

#### マイオペの質問文で

In [19]:
'''
    契約書を見たいのですが（正解＝4683）
'''
corpus, ret = predict('契約書を見たいのですが', dm=0)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が']
id, answer_id, similarity
3675_6 ('オットー・ハーン', '何が起きているのか意見を聞きたい」という手紙を送った') 0.43911951780319214
11747_9 ('制御工学', '詳細は下記を参照されたい') 0.42889660596847534
16025_4 ('待避駅', '信号場の記事を参照されたい') 0.4225197732448578
14004_9 ('絶版', 'しかし出版社と著者の間の契約が曖昧であったり、出版社が将来の人気再燃を睨んで出版権を保持しておきたがる場合もあるため、両者の区別が外部から見て判然としないことも少なくない') 0.4187547564506531
16285_4 ('双対', 'なお読みについて、双対を「そうたい」と読む流儀もあり「相対 (relative)」と紛らわしい') 0.4163917303085327
13599_0 ('電撃戦隊チェンジマン', '『電撃戦隊チェンジマン』（でんげきせんたいチェンジマン）は、1985年（昭和60年）2月2日から1986年（昭和61年）2月22日までテレビ朝日系列で毎週土曜18:00 - 18:25（JST）に全55話が放送された、東映制作の特撮テレビドラマ、および作中で主人公たちが変身するヒーローの名称') 0.41077715158462524
4998_3 ('ショウジョウバエ', 'それらに関してはキイロショウジョウバエの項を参照されたい') 0.40889424085617065
8506_1 ('ヤコブの手紙', '『ヤコブ書』とも') 0.40813201665878296
5226_0 ('メールマガジン', 'メールマガジン（和製英語:mail magazine）は、発信者が定期的にメールで情報を流し、読みたい人が購読するようなメールの配信の一形態') 0.4002209007740021
12921_2 ('四諦', '4つの諦（たい）を示す') 0.39957553148269653


In [20]:
'''
    EXカードを貸してください（正解＝4678）
'''
corpus, ret = predict('EXカードを貸してください', dm=0)
print_predict_result(corpus, ret)

['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる']
id, answer_id, similarity
1316_8 ('ワンダフルライフ (映画)', '選んでください') 0.5008060932159424
3815_4 ('都営地下鉄', 'パスネット対応カードをTカードの名称で取り扱っていた') 0.4984551966190338
10699_8 ('ロバート・タージャン', '高校のとき、パンチカードの照合の仕事を経験') 0.4798392951488495
17561_2 ('川崎市市民ミュージアム', '「TOP&カード提示割引」の加盟施設') 0.46452608704566956
15857_7 ('よかネットカード', '但し、無人駅や早朝・深夜などで駅員不在の場合はカードを使用できない') 0.4636368155479431
17098_6 ('近鉄吉野線', 'なお、2007年4月1日からサービスを開始したICカード「PiTaPa」「ICOCA」は全線で利用できる') 0.45559707283973694
1832_1 ('テーブルゲーム', '狭義のボードゲームおよびカードゲームなどを含む') 0.4514780640602112
1809_0 ('カードゲーム', 'カードゲームは、広義にはカードを使って行うゲームの総称') 0.45145124197006226
3125_9 ('御殿場線', '2010年3月13日から御殿場駅 - 沼津駅間にIC乗車カード「TOICA」が導入された') 0.44693076610565186
15859_5 ('トランパス (交通プリペイドカード)', 'カードを自動改札機に直接投入して入・出場する') 0.44338318705558777


In [21]:
'''
    一番目の質問の否定
'''
corpus, ret = predict('契約書を見せたくないのですが', dm=0)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見せる', 'たい', 'ない', 'の', 'です', 'が']
id, answer_id, similarity
3675_6 ('オットー・ハーン', '何が起きているのか意見を聞きたい」という手紙を送った') 0.46471646428108215
2812_4 ('可変電圧可変周波数制御', 'をそれぞれ参照の事') 0.4421604871749878
9311_9 ('宮藤官九郎', 'つーかたぶん177です」と語っている）') 0.42553505301475525
7041_9 ('数え上げ', '詳しくは、数え上げ数学を参照') 0.4188488721847534
2497_7 ('とんねるず', '1988年には『とんねるずのみなさんのおかげです』のレギュラー放送が開始') 0.41830605268478394
4091_4 ('日本の鉄道路線一覧', '参照：') 0.41207969188690186
6315_7 ('花婿失踪事件', 'だがその結婚式の朝、エンジェルは何か良くない事が起こりそうだという様子を見せる') 0.41188353300094604
10939_7 ('バークリウム', '物理的、化学的性質の詳細は良く分かっていない') 0.4109747111797333
16025_4 ('待避駅', '信号場の記事を参照されたい') 0.4100026488304138
17986_6 ('月 (オペラ)', 'それというのも、この世界が創造されたときに、夜のための光まで行き渡らなかったからです') 0.40527695417404175


#### 一番目の質問と言い回しを変えてみます。

別の予測結果が戻ります

In [22]:
'''
    一番目の質問の別な言い回し
'''
corpus, ret = predict('契約を結んだ時に作成される文書を確認したい', dm=0)
print_predict_result(corpus, ret)

['契約', 'を', '結ぶ', 'だ', '時', 'に', '作成', 'する', 'れる', '文書', 'を', '確認', 'する', 'たい']
id, answer_id, similarity
7004_9 ('目録', '官司でも国司の交替の際に正税などの在庫を確認する交替実録帳や班田実施時に作成する班田帳簿目録などが作られた') 0.4493039846420288
662_2 ('経営参加権', '共益権の一つであり、利益配当請求権・残余財産分配請求権などの自益権とともに、株主の権利の双璧をなす') 0.3900487720966339
4064_8 ('認証', '資格認証とは、何らかのサービスを受ける際にその資格があるかどうかを確認する為に行われる認証行為の事である') 0.38521042466163635
3924_1 ('乗車カード', '公共交通機関を利用する際、乗車時・降車時に読取機に情報を読み取らせるだけで運賃の支払いが可能なカードである') 0.3823910653591156
11452_7 ('磐梯町駅', '会津若松駅管理の簡易委託駅') 0.3809371590614319
13253_1 ('クレジットカード', '又は、契約者の番号その他が記載され、及び記録されたカードである') 0.3734629154205322
11841_6 ('社会保険', '保険料を主体としてできあがった財産を中心に一つの集団（保険集団）が組織され、保険集団の運営主体（保険者）と参加者（被保険者）の関係（保険関係）が生じる') 0.3724467158317566
7946_2 ('音源', '電子的に再生音を作成する機構、回路') 0.37100571393966675
1668_0 ('Debian', 'Debian（ デビアン）またはDebian ProjectはLinuxディストリビューションのひとつであるDebian GNU/Linuxを中心とするUnix系システムのディストリビューションを作成しているプロジェクトである') 0.3685429096221924
9650_9 ('シリコングラフィックス', '同日、Rackable Systems社による事業買収の合意が発表された') 0.3681261241436004

#### ラーメンに関する文書が戻ります

In [23]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
corpus, ret = predict('おいしいラーメンが食べたいです', dm=0)
print_predict_result(corpus, ret)

['おいしい', 'ラーメン', 'が', '食べる', 'たい', 'です']
id, answer_id, similarity
768_3 ('三枝義浩', '1989年、講談社の漫画雑誌『マガジンSPECIAL』掲載の『毎度ラーメン屋です』でデビュー') 0.4692099988460541
768_4 ('三枝義浩', '『毎度ラーメン屋です』はコメディタッチの作品だった') 0.4592877924442291
1348_2 ('藤原カムイ', '男性') 0.43731939792633057
531_1 ('池野恋', '岩手県花巻市出身、花巻市石鳥谷町在住') 0.43508124351501465
961_2 ('野中英次', '男性') 0.4349783658981323
596_1 ('内山まもる', '男性') 0.431552529335022
979_2 ('原秀則', '男性') 0.43116745352745056
3344_8 ('節句', 'ただ、今日でも人日の節句の七草粥など「節句料理」として残っているものがある') 0.43052196502685547
781_1 ('里見桂', '男性') 0.4300178587436676
14909_1 ('札幌ラーメン', 'ご当地ラーメン（ご当地グルメ）の一つ') 0.4291367828845978


#### 治療薬に関する文書が戻りました

In [24]:
'''
    無関係な質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=0)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
id, answer_id, similarity
15927_5 ('非ステロイド性抗炎症薬', '疼痛、発熱、炎症の治療に用いられる') 0.4261941611766815
7541_7 ('テオドール・エードラー・フォン・レルヒ', '10歳の時、福音学校に入学') 0.42226237058639526
2233_4 ('鈴木慶一', '弟はミュージシャンの鈴木博文') 0.42028141021728516
14935_1 ('湯通し', '湯をかけて、表面だけ加熱したい時、茹でて灰汁抜きする時、乾物を戻したりする時行う') 0.4183483123779297
585_8 ('加瀬邦彦', '所属していたホリプロはそれでも、加瀬を引きとめ給料を支払い続けていたが、「ただで給料をもらうのも申し訳ない」と同年7月に、加山雄三が名付け親となるザ・ワイルドワンズを結成（この時、渡辺プロに移籍）') 0.4155849814414978
1076_7 ('2組のお友達。', '彼女の運命はいかに') 0.40541744232177734
3720_3 ('遠藤周作', '帰国後の12歳の時に伯母の影響でカトリックの洗礼を受けた') 0.40512460470199585
13556_7 ('山本文緒', 'その後うつ病治療のため執筆活動を中断していたが、エッセイ『再婚生活』で復帰') 0.4032732844352722
8747_5 ('ナポレオン・ボナパルト', '出生時の洗礼名はナブリオーネ・ブオナパルテ') 0.40234190225601196
17073_4 ('抗凝固薬', '血栓塞栓症の予防、治療に対して用いる') 0.40201324224472046


#### 風邪についての文書が戻ります

In [25]:
'''
    Wikiタイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=0)
print_predict_result(corpus, ret)

['風邪']
id, answer_id, similarity
329_9 ('風邪', '「風邪」の語源は定かではない') 0.5652682185173035
2273_0 ('エレクトリック・ギター', 'エレクトリック・ギター（）は、ギターの一種') 0.5264266133308411
2499_2 ('セルフコンシステント', '撞着とは整合性がなく矛盾することを指し、無撞着であることは矛盾がなく整合することを意味する') 0.5178837776184082
10740_6 ('ビタミンC', 'しかし風邪を予防することはできない') 0.5166786909103394
6095_2 ('遺伝', 'しかし、生命現象としての遺伝は、後天的な母子感染による疾患や、非物質的情報伝達（学習など）による行動の類似化などを含まない') 0.5149567127227783
16130_4 ('蒲焼', '一般に「蒲焼」といえば「ウナギの蒲焼」を指すことが多く') 0.514083743095398
961_2 ('野中英次', '男性') 0.5136860609054565
11976_2 ('称名', '南無妙法蓮華経は題目と呼ばれ、称名にはあたらない') 0.5130983591079712
14801_0 ('太極拳', '太極拳（たいきょくけん）は、中国武術の一派') 0.5097821354866028
8639_3 ('文字', '「言語に直接結び付いて」とは、必ずしも文字が発音そのものを表すということではない') 0.5097341537475586


#### Wiki 文書の一部から抜粋した文章で予測

当たり前ですがドンピシャの予測結果となりました

In [26]:
'''
    Wiki本文に近い文言で、質問をしてみます。
'''
wiki_like_sentence = '''漁業にはさまざまな人々が関わっているが、漁撈活動に専業として携わる者のことは漁師という。
'''
corpus, ret = predict(wiki_like_sentence, dm=0)
print_predict_result(corpus, ret)

['漁業', 'に', 'は', 'さまざま', 'だ', '人々', 'が', '関わる', 'て', 'いる', 'が', '漁撈', '活動', 'に', '専業', 'として', '携わる', '者', 'の', 'こと', 'は', '漁師', 'と', 'いう']
id, answer_id, similarity
3321_3 ('漁業', '漁業にはさまざまな人々が関わっているが、漁撈活動に専業として携わる者のことは漁師という') 0.7201797366142273
17382_7 ('天津小湊町', '漁業') 0.47338032722473145
15060_8 ('プロフェッショナル', 'そして1961年には社会人野球の選手を引き抜いた（柳川事件）としてアマチュア野球との間に断絶が深まり、プロ野球経験者 (OB)がアマチュア野球へ関わることについて厳しい制限が課せられるようになった') 0.45838454365730286
1356_9 ('星野めみ', '実家が薬局であり、薬剤師として店にいることもある') 0.4515511393547058
4311_1 ('詩人', 'また、そのことを職業にしている者') 0.4463445246219635
17708_9 ('労働市場', '流動的な労働市場を形成し、新規の雇用を創出することが雇用のセーフティーネット拡充につながる」と指摘している') 0.4451509714126587
13984_1 ('星のカービィシリーズ', '桜井政博が生みの親で、彼が開発に関わった作品を「桜井カービィ」と呼ぶこともある') 0.4411381185054779
11286_7 ('ボーヤイ・ファルカシュ', 'ただ、ファルカシュはヤーノシュが数学者としての才能を持っていることに気づき、自ら息子に数学の教育を施した') 0.4340759217739105
5001_0 ('物理学者', '物理学者（ぶつりがくしゃ）は、物理学に携わる研究者のことである') 0.4266927242279053
10697_7 ('ジョン・バッカス', 'ニューヨーク市に移り住み、ラジオ技術者としての訓練を受け始め、数学が向いていることに気づいた') 0.42597782611846924


### (3-2) 適当な質問文で予測（PV-DM)

#### 全くもって性能が出なかったです。以下は参考データとしてご覧いただければ幸いです。

In [30]:
'''
    契約書を見たいのですが（正解＝4683）
'''
corpus, ret = predict('契約書を見たいのですが', dm=1)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見る', 'たい', 'の', 'です', 'が']
id, answer_id, similarity
516_6 ('秋月りす', '関西大学文学部国文学科卒') 0.40918096899986267
15403_8 ('普通乗車券', '特定都区市内を参照') 0.4015693664550781
8055_9 ('初台駅', '東口はオペラシティの営業時間に合わせて7時30分から23時30分まで利用できる') 0.40001821517944336
931_4 ('つの丸', '血液型はA型') 0.3922542631626129
648_3 ('高橋陽一', '血液型はA型') 0.39152178168296814
14985_0 ('日本共産党', '日本共産党（にほんきょうさんとう、）は、日本の政党') 0.3894280195236206
7321_4 ('石田純一', '血液型はA型') 0.3888404667377472
17668_1 ('片桐澪', '血液型はA型') 0.3884620666503906
3741_3 ('あゆみゆい', '血液型はA型') 0.388090044260025
15729_4 ('蒸気タービン', 'レシプロ型については蒸気機関を参照のこと') 0.38684558868408203


In [31]:
'''
    EXカードを貸してください（正解＝4678）
'''
corpus, ret = predict('EXカードを貸してください', dm=1)
print_predict_result(corpus, ret)

['ＥＸ', 'カード', 'を', '貸す', 'て', 'くださる']
id, answer_id, similarity
17626_1 ('91', ' ') 0.19188746809959412
14523_1 ('34', ' ') 0.16251426935195923
5606_2 ('7月17日', ' ') 0.1248469203710556
5593_2 ('7月29日', ' ') 0.12462043762207031
5102_2 ('6月26日', ' ') 0.1153639554977417
16685_1 ('69', ' ') 0.11440088599920273
4895_2 ('5月6日', ' ') 0.10737568885087967
1078_6 ('必ずお読み下さい。', 'あるいは「裏ぶたを開けないでください') 0.10550728440284729
10637_5 ('自然対数', '自然対数は、任意の正数 に対して 逆数函数 の から までの間のグラフの下にある面積（ のときは負の面積）として定義することもできる') 0.1010487750172615
14752_7 ('素因数分解', 'この右辺から分かることは、360 の正の約数はの形をしていて、指数はの範囲に限られるということである') 0.10085368156433105


In [32]:
'''
    一番目の質問の否定
'''
corpus, ret = predict('契約書を見せたくないのですが', dm=1)
print_predict_result(corpus, ret)

['契約', '書', 'を', '見せる', 'たい', 'ない', 'の', 'です', 'が']
id, answer_id, similarity
864_1 ('柴田亜美', '血液型はB型') 0.43204087018966675
1291_3 ('中川いさみ', '血液型はB型') 0.42943447828292847
2592_7 ('湯川秀樹', '位階勲等は従二位勲一等旭日大綬章') 0.4291611313819885
11005_1 ('八千代緑が丘駅', '駅番号はTR06') 0.42904597520828247
8703_1 ('木村千歌', '血液型はB型') 0.42789411544799805
3295_2 ('矢口史靖', '東京造形大学卒業') 0.4235425591468811
16752_1 ('K', '小文字は k') 0.42292720079421997
491_3 ('高河ゆん', '血液型はB型') 0.4226906895637512
1851_2 ('井上雄彦', '血液型はB型') 0.4226759076118469
337_1 ('あずまきよひこ', 'よつばスタジオ社員') 0.42249464988708496


In [33]:
'''
    一番目の質問の別な言い回し
'''
corpus, ret = predict('契約を結んだ時に作成される文書を確認したい', dm=1)
print_predict_result(corpus, ret)

['契約', 'を', '結ぶ', 'だ', '時', 'に', '作成', 'する', 'れる', '文書', 'を', '確認', 'する', 'たい']
id, answer_id, similarity
516_6 ('秋月りす', '関西大学文学部国文学科卒') 0.496515691280365
12697_1 ('ガミラス帝国', '地球人類に対する敵対勢力') 0.48960191011428833
939_1 ('ともち', '神奈川県横浜市出身') 0.4800870716571808
3295_1 ('矢口史靖', '神奈川県伊勢原市出身') 0.4799981713294983
871_1 ('新谷かおる', '大阪府豊中市出身') 0.4771193563938141
1388_1 ('望月峯太郎', '神奈川県横浜市出身') 0.47324904799461365
14195_1 ('おもちゃのまち駅', '駅番号はTN 35') 0.4730132222175598
763_1 ('近藤和久', '愛知県豊田市出身') 0.470525860786438
13080_1 ('2027年', '注釈出典') 0.4703509211540222
18272_1 ('神谷明', '神奈川県横浜市出身') 0.46941301226615906


In [34]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
corpus, ret = predict('おいしいラーメンが食べたいです', dm=1)
print_predict_result(corpus, ret)

['おいしい', 'ラーメン', 'が', '食べる', 'たい', 'です']
id, answer_id, similarity
17525_3 ('村上ゆみ子', '血液型はAB型') 0.6796249747276306
17668_1 ('片桐澪', '血液型はA型') 0.6739862561225891
7321_4 ('石田純一', '血液型はA型') 0.6731369495391846
648_3 ('高橋陽一', '血液型はA型') 0.6704878211021423
758_2 ('小谷憲一', '血液型はAB型') 0.6691522002220154
2228_2 ('わたべ淳', '血液型はAB型') 0.6672954559326172
647_3 ('佐々木倫子', '血液型はAB型') 0.6662349104881287
730_3 ('川原正敏', '血液型はA型') 0.6642611026763916
252_3 ('山下和美', '血液型はAB型') 0.6641671657562256
966_4 ('のむらしんぼ', '血液型はAB型') 0.6622813940048218


In [35]:
'''
    無関係な質問をしてみます。
'''
corpus, ret = predict('風邪を引いた時の治療薬を教えてください', dm=1)
print_predict_result(corpus, ret)

['風邪', 'を', '引く', 'た', '時', 'の', '治療', '薬', 'を', '教える', 'て', 'くださる']
id, answer_id, similarity
17005_0 ('ヘパリン', 'ヘパリン (heparin) は抗凝固薬の一つであり、血栓塞栓症や播種性血管内凝固症候群 (DIC) の治療、人工透析、体外循環での凝固防止などに用いられる') 0.2738163471221924
10564_2 ('香具師', '古くは、香具師（こうぐし）とも読み、主に江戸時代では歯の民間治療をしていた辻医者（つじいしゃ）や、軽業・曲芸・曲独楽などをして客寄せをし、薬や香具を作ったり、売買していた露天の商売人を指した') 0.2647455632686615
5226_1 ('メールマガジン', 'MM、メルマガと略されることも多い') 0.2603487968444824
10295_6 ('二銭銅貨', 'そしてこの二銭銅貨の秘密に気がついた松村は、これをきっかけに一人で捜査を始め、ついに盗まれた5万円の行方をつきとめたとして、「私」に得意げにその謎を解き明かしてみせる') 0.25105515122413635
6673_5 ('光電効果', 'この現象を外部光電効果、または単に光電効果と言う') 0.2475043535232544
4069_3 ('電気工学', 'この観点から、前者を「強電」、後者を「弱電」と二分される') 0.2450515627861023
13428_3 ('ポジトロン断層法', '患者への被曝量はCTに比べて少ないが、医療スタッフの被曝量に注意が必要である') 0.23992541432380676
6162_1 ('陽子', '英語名のままプロトンと呼ばれることも多い') 0.23889189958572388
16517_8 ('音楽療法', '治療効果も古くから知られ、ダビデはサウルのうつ病を竪琴で治したとされる（旧約聖書『サムエル記』上16.14–23）') 0.23805618286132812
3845_7 ('カバ', 'しかし、DNA分析の結果、カバはイノシシ類よりもウシと近縁であることが証明され、かつ、クジラとの遺伝的関係が最も近い陸上動物であることが分かった') 0.236598283

In [36]:
'''
    Wikiタイトルに近い文言で、Google検索っぽく質問をしてみます。
'''
corpus, ret = predict('風邪', dm=1)
print_predict_result(corpus, ret)

['風邪']
id, answer_id, similarity
2228_2 ('わたべ淳', '血液型はAB型') 0.5971587896347046
17525_3 ('村上ゆみ子', '血液型はAB型') 0.5913268327713013
252_3 ('山下和美', '血液型はAB型') 0.5857034921646118
1111_2 ('成田美名子', '血液型はAB型') 0.5846182107925415
1294_2 ('松下容子', '血液型はAB型') 0.5824437141418457
966_4 ('のむらしんぼ', '血液型はAB型') 0.580870509147644
15048_6 ('小池一夫', '中央大学法学部卒') 0.5808088779449463
647_3 ('佐々木倫子', '血液型はAB型') 0.5781458616256714
2621_1 ('3世紀', '注釈出典') 0.5780136585235596
900_5 ('井上陽水', '血液型はAB型') 0.5779861211776733


In [37]:
'''
    Wiki本文に近い文言で、質問をしてみます。
'''
wiki_like_sentence = '''漁業にはさまざまな人々が関わっているが、漁撈活動に専業として携わる者のことは漁師という。
'''
corpus, ret = predict(wiki_like_sentence, dm=1)
print_predict_result(corpus, ret)

['漁業', 'に', 'は', 'さまざま', 'だ', '人々', 'が', '関わる', 'て', 'いる', 'が', '漁撈', '活動', 'に', '専業', 'として', '携わる', '者', 'の', 'こと', 'は', '漁師', 'と', 'いう']
id, answer_id, similarity
11885_1 ('紀元前6世紀', '注釈出典') 0.7933207750320435
2621_1 ('3世紀', '注釈出典') 0.7917889356613159
4215_1 ('1750年代', '注釈出典') 0.7910169363021851
8412_5 ('1世紀', '注釈出典') 0.7902028560638428
17525_3 ('村上ゆみ子', '血液型はAB型') 0.789082407951355
4479_2 ('紀元前3千年紀', '注釈出典') 0.7870775461196899
4476_1 ('紀元前25世紀', '注釈出典') 0.7859131097793579
1922_1 ('1972年', '注釈出典') 0.7848941087722778
6053_2 ('2040年代', '注釈出典') 0.7848808169364929
1938_1 ('1955年', '注釈出典') 0.7844297885894775
